In [1]:
from pathlib import Path
import numpy as np
import os

In [13]:
class LearnVocabulary:
    def __init__(self, path2txt: str, alias: str) -> None:
        self.path2txt = path2txt
        self.name = alias
        self.load_txt()
        self.preprocessing_name_of_language()
        self.preprocessing_text()
    
    def load_txt(self) -> None:
        with open(self.path2txt, encoding='utf-8') as f:
            self.list_of_sentence = f.readlines()
            self.name_of_language = self.list_of_sentence[0]
            self.list_of_sentence = self.list_of_sentence[1:]

    def preprocessing_text(self) -> None:
        list_of_word1 = []
        list_of_word2 = []
        for sentence in self.list_of_sentence:
            split_sentence = sentence.split('-')
            list_of_word1.append(split_sentence[0].strip())
            list_of_word2.append(split_sentence[-1].strip())
        self.language1 = {word1: word2 for word1, word2, in zip(list_of_word1, list_of_word2)}
        self.language2 = {word2: word1 for word1, word2, in zip(list_of_word1, list_of_word2)}

    def preprocessing_name_of_language(self) -> None:
        self.language = self.name_of_language.split('-')
        self.language = [lang.strip() for lang in self.language]


    def present_dictionary(self, option: int=1) -> None:
        '''
        Presentation of all words in the tested dictionary.
        :param: option: int=1 --> which language should be in the first column? (1- first, 2-second, another-error)
        '''
        print('##### All words in dictionary #####')
        if(option==1):
            idx = 1
            print(f'#### {self.language[0]} -- {self.language[1]}')
            for key, item in zip(self.language1.keys(), self.language1.values()):
                print(f'## {idx}. {key} -- {item}')
                idx += 1
        elif(option==2):
            idx = 1
            print(f'#### {self.language[1]} -- {self.language[0]}')
            for key, item in zip(self.language1.keys(), self.language1.values()):
                print(f'## {idx}. {key} -- {item}')
                idx += 1
        else:
            print('Error: option must be {1,2}')
    
    def create_learning_list(self) -> None:
        basic_path = self.path2txt[:-4]
        self.path2lang1 = basic_path + f'_ProgressList_{self.language[0]}_{self.name}.txt'
        self.path2lang2 = basic_path + f'_ProgressList_{self.language[1]}_{self.name}.txt'
        if Path(self.path2lang1).is_file():
            print('Your vocabulary list does exist!')
        else:
            print("Your vocabulary list doesn't exist! We create it")
            with open(self.path2lang1, 'w', encoding='utf-8') as f:
                for item in self.language1.keys():
                    f.write("%s\n" % item)
        if Path(self.path2lang2).is_file():
            print('Your vocabulary list does exist!')
        else:
            print("Your vocabulary list doesn't exist! We create it")
            with open(self.path2lang2, 'w', encoding='utf-8') as f:
                for item in self.language2.keys():
                    f.write("%s\n" % item)

    def load_your_vocabulary_list(self) -> None:
        with open(self.path2lang1, encoding='utf-8') as f:
            self.vocabulary_lang1_list = f.readlines()
            self.vocabulary_lang1_list = [word.strip().replace('\n', '') for word in self.vocabulary_lang1_list]
        with open(self.path2lang2, encoding='utf-8') as f:
            self.vocabulary_lang2_list = f.readlines()
            self.vocabulary_lang2_list = [word.strip().replace('\n', '') for word in self.vocabulary_lang2_list]

    def update_progress_list(self, language_task: int=1):
        if(language_task==1):
            os.remove(self.path2lang1)
            with open(self.path2lang1, 'w', encoding='utf-8') as f:
                for item in self.vocabulary_lang1_list:
                    f.write("%s\n" % item)
        elif(language_task==2):
            os.remove(self.path2lang2)
            with open(self.path2lang2, 'w', encoding='utf-8') as f:
                for item in self.vocabulary_lang2_list:
                    f.write("%s\n" % item)


    def rules(self) -> None:
        print("""!RULES!
        You will get n words to translate in both languages. 
        Your progress list will be saved after each task. 
        If you translate the word correctly, it will be removed word from your progress list. 
        If you translate a word incorrectly, the word will be added to your progress list.""")
        print("Let's get started")

    def check_answer(self, word, input_word, language_task: int=1) -> bool:
        if (language_task==1):
            if(self.language1[word] == input_word):
                return True
            else:
                return False
        else: 
            if(self.language2[word] == input_word):
                return True
            else:
                return False

    def remove_word_from_progress_list(self, word: str, vocabulary_list: list, language_task: int=1) -> list:
        idx = vocabulary_list.index(word)
        vocabulary_list.remove(vocabulary_list[idx])
        return vocabulary_list

    def add_word_to_progress_list(self, word: str, vocabulary_list: list, language_task: int=1) -> list:
        idx = vocabulary_list.index(word)
        vocabulary_list.append(vocabulary_list[idx])
        return vocabulary_list
    
    def translation_task(self, num_of_word: int, language_task: int=1) -> None:
        num_of_correct_answer = 0
        if(language_task==1):
            if(len(self.vocabulary_lang1_list) < num_of_word):
                num_of_word = len(self.vocabulary_lang1_list)
            list_of_word = np.random.choice(self.vocabulary_lang1_list, num_of_word, replace=False)
            # print(self.vocabulary_lang1_list)
            for idx, word in enumerate(list_of_word):
                print(f"# {idx}. {word} --> ")
                input_word = str(input(f"# {idx}. {word} --> ")).strip()
                if(self.check_answer(word, input_word, language_task)):
                    print(f"### -----> {input_word} -- Correct")
                    self.vocabulary_lang1_list = self.remove_word_from_progress_list(word, self.vocabulary_lang1_list, language_task)
                    num_of_correct_answer += 1
                else:
                    print(f"### -----> {input_word} -- Wrong (must be '{self.language1[word]}')")
                    self.vocabulary_lang1_list = self.add_word_to_progress_list(word, self.vocabulary_lang1_list, language_task)
            self.update_progress_list(language_task) 
        else:
            if(len(self.vocabulary_lang2_list) < num_of_word):
                num_of_word = len(self.vocabulary_lang2_list)
            list_of_word = np.random.choice(self.vocabulary_lang2_list, num_of_word, replace=False)
            for idx, word in enumerate(list_of_word):
                print(f"# {idx}. {word} --> ")
                input_word = str(input(f"# {idx}. {word} --> ")).strip()
                if(self.check_answer(word, input_word, language_task)):
                    print(f"### -----> {input_word} -- Correct")
                    self.vocabulary_lang2_list = self.remove_word_from_progress_list(word, self.vocabulary_lang2_list, language_task)
                    num_of_correct_answer += 1
                else:
                    print(f"### -----> {input_word} -- Wrong (must be '{self.language1[word]}')")
                    self.vocabulary_lang2_list = self.add_word_to_progress_list(word, self.vocabulary_lang2_list, language_task)
            self.update_progress_list(language_task) 
        print(f'########## Your score: {num_of_correct_answer}/{num_of_word} ({100*num_of_correct_answer/num_of_word}%)')
        


    def start_testing(self, num_of_word: int) -> None:
        self.create_learning_list()
        self.load_your_vocabulary_list()
        self.rules()
        self.translation_task(num_of_word)
        #TODO Podział na zadania

        # input_word = str(input("Translate the word: "))
        # print("to jest twoja liczba", a)

In [14]:
lv = LearnVocabulary(path2txt='vocabulary\wladaj_i_gadaj_adv_1.txt', alias='MM')
# lv.load_txt()
# lv.present_dictionary()
lv.start_testing(2)
# list(lv.list_of_sentence)

Your vocabulary list does exist!
Your vocabulary list does exist!
!RULES!
        You will get n words to translate in both languages. 
        Your progress list will be saved after each task. 
        If you translate the word correctly, it will be removed word from your progress list. 
        If you translate a word incorrectly, the word will be added to your progress list.
Let's get started
# 0. fetch --> 
### -----> przynieść -- Correct
# 1. discover --> 
### -----> odkryć -- Correct
########## Your score: 2/2 (100.0%)


In [21]:
lv.vocabulary_lang1_list.index('castaway')

3

In [26]:
kkk = ['a','a','b']
kkk.remove(kkk[kkk.index('b')])

In [27]:
kkk

['a', 'a']